In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate, Dropout
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.utils import to_categorical

from datetime import datetime
import os
import glob
import shutil
import random
import cv2
# from google.colab import drive
import zipfile

In [2]:
# # 관리할 폴더 미리 지정해주기(만들어주는게 아님)

# DATA_ROOT_DIR = os.path.join("c:/han/food_file", "food")
# food_file_list = "C:\\han\\food_file\\food/train"
# label_name_list = os.listdir(food_file_list)
# TRAIN_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, "train") # train이라는 폴더를 만들어 주겠다.
# TEST_DATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR, "test")
# # label_name_list = 폴더이름이자 정답이름


# 이미지 저장할 폴더 준비


DATA_ROOT_DIR = "c:/han/food_file/food/train"
label_name_list = os.listdir(DATA_ROOT_DIR)

# label_name_list = 폴더이름이자 정답이름

In [23]:
# # 압축 풀어주기 (풀면 파일이 만들어짐)
# with zipfile.ZipFile(os.path.join("c:/han/food_file", "food.zip"), "r") as target_file:
#     target_file.extractall(DATA_ROOT_DIR)  



# 압축 풀어놓기
with zipfile.ZipFile("c:/han/food_file/food.zip", "r") as target_file:
    target_file.extractall("c:/han/food_file/food/train/")

In [3]:
label_name_list

['Burger', 'chicken', 'croffle', 'pasta', 'Pizza', 'pork cutlet', 'tteokbokki']

In [9]:
# 풀어놓은 전체 데이터를 TRAIN파일에 옮겨주자

shutil.copytree(DATA_ROOT_DIR, TRAIN_DATA_ROOT_DIR) # COPYTREE는 기존에 똑같은 이름의 폴더가 있으면 에러가 생김.

'c:/han/food_file\\food\\train'

In [25]:
# # TEST폴더 만들어주자 / 기존에 같은 명의의 파일이 있으면 에러생김
# os.mkdir(TEST_DATA_ROOT_DIR)
# os.mkdir(os.path.join(TEST_DATA_ROOT_DIR, "Burger"))
# os.mkdir(os.path.join(TEST_DATA_ROOT_DIR, "Pizza"))
# os.mkdir(os.path.join(TEST_DATA_ROOT_DIR, "chicken"))
# os.mkdir(os.path.join(TEST_DATA_ROOT_DIR, "croffle"))
# os.mkdir(os.path.join(TEST_DATA_ROOT_DIR, "pasta"))
# os.mkdir(os.path.join(TEST_DATA_ROOT_DIR, "pork cutlet"))
# os.mkdir(os.path.join(TEST_DATA_ROOT_DIR, "tteokbokki"))

# test데이터폴터 만들어주기

if not os.path.exists("c:/han/food_file/food/test"):
    os.mkdir("c:/han/food_file/food/test")
    print("c:/han/food_file/food/test directory is created")
    
    count = 0
    for label_name in label_name_list:
        os.mkdir("c:/han/food_file/food/test/" + label_name.strip())
        print("c:/han/food_file/food/test/" + label_name.strip() + " is created")
        
        count = count + 1
        
    print("총 횟수 : ", count)

c:/han/food_file/food/test directory is created
c:/han/food_file/food/test/Burger is created
c:/han/food_file/food/test/chicken is created
c:/han/food_file/food/test/croffle is created
c:/han/food_file/food/test/pasta is created
c:/han/food_file/food/test/Pizza is created
c:/han/food_file/food/test/pork cutlet is created
c:/han/food_file/food/test/tteokbokki is created
총 횟수 :  7


In [13]:
label_name_list

['Burger',
 'chicken',
 'croffle',
 'pasta',
 'Pizza',
 'pork cutlet',
 'test',
 'train',
 'tteokbokki']

In [48]:
ratio = 0.2

src_root_dir = "c:/han/food_file/food/train/"
dst_root_dir = "c:/han/food_file/food/test/"

for label_name in label_name_list:
    train_image_file_list = glob.glob(src_root_dir + label_name + "/*.*")

#     print(src_root_dir + label_name)
#     print("C:/han/food_file/food/train/Burger")
    print("==========================================================")
    
    print("total [%s] image file nums => [%s]"%(label_name, len(train_image_file_list)))
    
    split_num = int(ratio * len(train_image_file_list))
    print("분리할 갯수 : ", split_num)
    
    test_image_file_list = train_image_file_list[0:split_num]
    
    # move
    move_nums = 0
    for image_file in test_image_file_list:
        shutil.move(image_file, dst_root_dir + label_name)
        move_nums = move_nums + 1
        
    print("total move nums => ", move_nums)
    print("===========================================================")

total [Burger] image file nums => [1500]
분리할 갯수 :  300
total move nums =>  300
total [chicken] image file nums => [1396]
분리할 갯수 :  279
total move nums =>  279
total [croffle] image file nums => [1267]
분리할 갯수 :  253
total move nums =>  253
total [pasta] image file nums => [1230]
분리할 갯수 :  246
total move nums =>  246
total [Pizza] image file nums => [1500]
분리할 갯수 :  300
total move nums =>  300
total [pork cutlet] image file nums => [1208]
분리할 갯수 :  241
total move nums =>  241
total [tteokbokki] image file nums => [1549]
분리할 갯수 :  309
total move nums =>  309


In [4]:
##### 검증 코드

src_root_dir = "c:/han/food_file/food/train/"
dst_root_dir = "c:/han/food_file/food/test/"

train_label_name_list = os.listdir(src_root_dir)
test_label_name_list = os.listdir(dst_root_dir)

train_label_name_list.sort()
test_label_name_list.sort()

if train_label_name_list != test_label_name_list:
    print("fatal error !")
    
else:
    print(len(train_label_name_list), len(test_label_name_list))

7 7


In [5]:
##### TRAIN 형식 맞춰주기

image_list = []
label_list = []

train_base_dir = "c:/han/food_file/food/train/"
train_label_list = os.listdir(train_base_dir)


start_time = datetime.now()
for label_name in train_label_list:
    file_path = train_base_dir + label_name
    print(file_path)
    img_file_list = glob.glob(file_path + "/*.*")
    
    
    for img_file in img_file_list:
        src_img = cv2.imread(img_file, cv2.IMREAD_COLOR)
        src_img = cv2.resize(src_img, dsize=(128,128))
        src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
        
        image_list.append(src_img)        
        label_list.append(label_list)

        
end_time = datetime.now()
print("Elapsed Time : ", end_time - start_time)

c:/han/food_file/food/train/Burger
c:/han/food_file/food/train/chicken
c:/han/food_file/food/train/croffle
c:/han/food_file/food/train/pasta
c:/han/food_file/food/train/Pizza
c:/han/food_file/food/train/pork cutlet
c:/han/food_file/food/train/tteokbokki
Elapsed Time :  0:00:39.705383


In [6]:
image_list[1]

array([[[ 54,  61,  77],
        [ 43,  49,  63],
        [ 33,  39,  53],
        ...,
        [ 48,  57,  64],
        [110, 115, 120],
        [ 19,  19,  21]],

       [[ 58,  65,  80],
        [ 45,  51,  65],
        [ 34,  40,  54],
        ...,
        [ 38,  46,  52],
        [ 47,  49,  54],
        [ 26,  26,  28]],

       [[ 62,  69,  85],
        [ 48,  54,  68],
        [ 35,  41,  55],
        ...,
        [  3,   5,   8],
        [  2,   3,   7],
        [  5,   5,   7]],

       ...,

       [[102,  91,  82],
        [ 99,  89,  80],
        [101,  92,  84],
        ...,
        [110,  82,  71],
        [112,  82,  72],
        [110,  82,  70]],

       [[ 40,  30,  24],
        [ 37,  30,  24],
        [ 26,  20,  16],
        ...,
        [108,  80,  69],
        [111,  81,  70],
        [112,  84,  72]],

       [[ 16,  10,   6],
        [ 13,   9,   5],
        [  9,   5,   3],
        ...,
        [109,  81,  70],
        [111,  81,  71],
        [111,  83,  71]]

In [ ]:
X_train = np.array(image_list)
y_train = np.array(label_list)
X_train.shape

In [ ]:
##### TEST

image_list = []
label_list = []

test_base_dir = "c:/han/food_file/food/test/"
test_label_list = os.listdir(test_base_dir)


strat_time = datetime.now()
for label_name in test_label_list:
    file_path = test_base_dir + label_name
    img_file_list = glob.glob(file_path + "/*.*")
    
    for img_file in img_file_list:
        src_img = cv2.imread(img_file, cv2.IMREAD_COLOR)
        src_img = cv2.resize(src_img, dsize=(128,128))
        src_img = cv2.cvtColor(src_img, cv2.COLOR_BGR2RGB)
        
        image_list.append(src_img)
        label_list.append(label_name)
        
end_time = datetime.now()
print("Elapsed Time : ", end_time - start_time)